# <font color=Crimson size=6 face="宋体" align="center">二维平面坐标单层感知器线性划分</font>     
--------------

<font color=black size=4 face="微软雅黑" align="center">单层感知器模型如下图:</font> 

<img src="./perceptron.png" align="left">       





 <font color=black size=4 face="微软雅黑" align="left">人工神经元是一个多输入、单输出的非线性元件，其输入与输出关系可以表示为:</font> 

<font color=#0000FF size=4 face="微软雅黑" align="center">$$y=f(\sum_{i=1}^{n}\omega_i*x_i+b)$$</font>   

<font color=#purple size=4 face="微软雅黑" align="left">其中$x_1,x_2,\cdots,x_n$为训练样本的n个属性，$\omega_1,\omega_2，\cdots,\omega_n$为针对每一个属性的权重，b为偏置信号，f为激活函数</font>     



------------

---------
*********
# 接下来就以2维平面坐标为例，使用单层感知器进行分类
 ## 1.首先导入输入训练样本:10个2维坐标
 

In [1]:
import numpy as np
#定义坐标,设定10组输入数据，每组为（x,y）
Training_Input=np.array([[1,4],
            [3,5],
            [8,10],    
            [2,8],            
            [4,9],
            [7,6],            
            [1,1],  
            [7,5],
            [2,2],
            [3,4]]);
new_col=np.ones((1,len(Training_Input)))      #生成一列偏置信号全为1     
Training_Input= np.column_stack((Training_Input,new_col.T))      #将偏置信号加入训练样本中
print("训练样本矩阵如下：")
Training_Input

训练样本矩阵如下：


array([[ 1.,  4.,  1.],
       [ 3.,  5.,  1.],
       [ 8., 10.,  1.],
       [ 2.,  8.,  1.],
       [ 4.,  9.,  1.],
       [ 7.,  6.,  1.],
       [ 1.,  1.,  1.],
       [ 7.,  5.,  1.],
       [ 2.,  2.,  1.],
       [ 3.,  4.,  1.]])

  
  ## 2.针对每一个训练样本都给出它的期望输出

In [2]:
import matplotlib.pyplot as plt
#设定输入向量的期待输出值
Target_Output=np.array([1,1,1,1,1,-1,-1,-1,-1,-1]);
for i in range(len(Training_Input)):
    if(Target_Output[i]==1):
        plt.plot(Training_Input[i,0],Training_Input[i,1],'bo')
    else:
        plt.plot(Training_Input[i,0],Training_Input[i,1],'ro')

print("输入的训练样本集如下图所示：")
plt.show()

输入的训练样本集如下图所示：


<Figure size 640x480 with 1 Axes>

 
  
  ## 3.在迭代之前，对权重初始化

In [3]:
#设定权值向量(w0,w1,w2),权值范围为-1,1
weights = (np.random.random(3)-0.5)*2;
weights_initial=weights 
print("初始值：       w0          w1           w2")
weights

初始值：       w0          w1           w2


array([ 0.94367633, -0.30905472, -0.75385746])

## 4.设定学习率，记录感知器的迭代次数，初始化感知器的输出

In [4]:
#设定学习率
learning_rate = 0.3;
#计算迭代次数
iterative_num=0;
#神经网络输出
Output=0;

## 5.对感知器进行训练
  ### 在这个过程激活函数采用的是阶跃函数，表达式为：
$$f(x)=\begin{cases}
1 & (x\geq0) \\
0 & (x<0)
\end{cases}$$

### 每一次迭代后，根据感知器的输出调整各权重($\omega_1,\omega_2，\cdots,\omega_n$)的大小   
   
$$\Delta\omega=(y'-y)*\eta*x$$
 
$$\omega_(new)=\omega_(old)+\Delta\omega$$   
 
  ### 其中y'为期望输出，y为当前迭代感知器的输出，$\eta$为学习率

In [5]:
def  update():
    global  Training_Input,Target_Output,weights,learning_rate,iterative_num;   #引用的全局变量
    iterative_num=iterative_num+1;      #每迭代一次，迭代次数+1
    f_out=np.sign(np.dot(Training_Input,weights.T));    #通过阶跃函数处理后作为感知器的输出
    #计算权值差
    W_Tmp = learning_rate*((Target_Output-f_out.T).dot(Training_Input));
    weights = weights+W_Tmp;

In [6]:
weights_stored=weights
MAX_TRAINING=5000     #设置最大的训练次数，超过这个次数就停止训练
linear_separable=True
while(1):
    update()
    Output=np.sign(np.dot(Training_Input,weights.T))
    weights_stored=np.row_stack((weights_stored,weights))      #将每一次迭代后weight的值保存起来，一行一行地加入weights_stored矩阵
    if(Output==Target_Output).all():
        print('训练完成！')
        print('epoch:',iterative_num)
        linear_separable=True
        break
    if(iterative_num>MAX_TRAINING):
        print("输入的训练样本集好像不是线性可分的，我已训练"+str(MAX_TRAINING)+"次了！  不过再训练更多次也许能够做到哦！")
        linear_separable=False
        break

训练完成！
epoch: 42


## 6.展示训练结果（通过移动滑条来查看不同迭代次数得到的分类直线）

In [7]:
from pylab import *                                            #导入中文包
mpl.rcParams['font.sans-serif'] = ['SimHei']                   #设置中文字体，如果不需要在图上显示中文，这两行大可不必要
from ipywidgets import interact, interactive, fixed, interact_manual
@interact(Iterative_times=(0,len(weights_stored)-1,1))
def update_show_2(Iterative_times):
   #分类直线y=kx+b
    k=-weights_stored[Iterative_times,0]/weights_stored[Iterative_times,1]     #计算分类直线的k
    d=-weights_stored[Iterative_times,2]/weights_stored[Iterative_times,1]     #计算分类直线的b
    xdata=np.linspace(0,10)
    plt.figure()
    
    for i in range(len(Training_Input)):                       #在图上画出训练样本点
        if(Target_Output[i]==1):
            plt.plot(Training_Input[i,0],Training_Input[i,1],'bo')
        else:
            plt.plot(Training_Input[i,0],Training_Input[i,1],'ro')

    plt.plot(xdata,xdata*k+d,'g')                              #在图上画出分类直线
    plt.title('训练'+str(Iterative_times)+'次的分类直线')      #给图增加标题
    plt.show()
    
    print("   w0          w1           w2")             
    print(weights_stored[Iterative_times,0:])                 #显示此时的权值
    #判断滑条滑至最右端的情况
    if(Iterative_times==len(weights_stored)-1 and linear_separable==True):
        print("感知器训练已结束，权值将不会再发生改变！！！")
    elif(Iterative_times==len(weights_stored)-1 and linear_separable==False):
        print("它们似乎不是线性可分的")


interactive(children=(IntSlider(value=21, description='Iterative_times', max=42), Output()), _dom_classes=('wi…